In [174]:
import csv
import numpy as np
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
import math

In [175]:
#############
## DATA IO ##
#############

def get_data(filepath):
    # Opens the file handler for the dataset file. Using variable 'f' we can access and manipulate our file anywhere 
    # in our code after the next code line.
    f = open(filepath,"r")

    # Predictors Collection (or your input variable) (which in this case is the Sepal length, Sepal width, Petal length and Petal width)
    X = [[],[],[],[]]

    # Output Response (or your output variable) (which in this case is the category to which the iris flower belongs.
    # Remember: We are interested in seperating Setosa category [label - 1] from other non-setosa categories [label - 0].)
    Y = []

    # Initializing a reader generator using reader method from csv module. A reader generator takes each line from the 
    # file and converts it into list of columns.
    reader = csv.reader(f)

    # Using for loop, we are able to read one row at a time.
    for row in reader:
        # For extracting out 4 input variables i.e Sepal length, Sepal width, Petal length, Petal width
        for i in range(0,4):
            X[i].append(float(row[i]))
        
        # For extracting output category (Y) i.e 1 for Setosa and 0 for Non-Setosa
        if row[4] == "Iris-setosa":
            Y.append(1)
        else:
            Y.append(0)

    # Close the file once we have succesffuly stored all data into our X and Y variables.
    f.close()
    
    # Normalization of Input Data
    mean=[]
    
    std=[]
    
    for i in range(0,4):
        mean.append(np.mean(X[i]))
        std.append(np.std(X[i]))

    return [np.array([(X[i] - mean[i])/(std[i]) for i in range(0,4)]),np.array(Y),mean,std]


# Function for normalized output
def normalize_for_output(x,mean,std):
    return [(x[i] - mean[i])/(std[i]) for i in range(0,4)]

In [176]:
def logistic(a):
    return (1/(1+np.exp(-1*a)))

In [177]:
#####################
## Error Calculation ##
#####################

def error(x, y, betas):
    error = 0
    for i in range(x[0].shape[0]):
        predicted_value = logistic(betas[0] + (betas[1] * x[0][i])+(betas[2] * x[1][i])+(betas[3] * x[2][i])+(betas[4] * x[3][i]))
        actual_value = y[i]
        error = error + ((-predicted_value*np.log(predicted_value))-((1-actual_value)*np.log(1-predicted_value)))
    return (error)

In [178]:
def predicted_value_final(X,i,betas):
    return (logistic(betas[0] + (betas[1] * X[0][i]) + (betas[2] * X[1][i]) + (betas[3] * X[2][i]) + (betas[4] * X[3][i])))

In [179]:
def gradientDescentAlgorithm(x, y, learning_rate):
    
    print ("Training Linear Regression Model using Gradient Descent")
    
    maximum_iterations = 100
    
    # This flag lets the program know wether the gradient descent algorithm has reached it's converged state which means wether 
    # the algorithm was able to find the local minima (where the slope of RSS wrt your parameters beta_0 and beta_1 is zero)
    converge_status = False
    
    # num_rows stores the number of datapoints in the current dataset provided for training.
    num_rows = x[0].shape[0]

    # Initial Value of parameters 
    betas = [0,0,0,0,0]
    
    # Initial Error or RSS(beta_0,beta_1) based on the initial parameter values
    #error = RSS(x, y, beta_0, beta_1)
    _error = error(x, y, betas)
    #print('Initial Value (Cost Function)=', error);
    
    # Iterate Loop
    num_iter = 0
    while not converge_status:
        # for each training sample, compute the gradient (d/d_beta j(beta))
        gradient_0 = 1.0/num_rows * sum([(predicted_value_final(x,i,betas) - y[i]) for i in range(num_rows)]) 
        gradient_1 = 1.0/num_rows * sum([(predicted_value_final(x,i,betas) - y[i])*x[0][i] for i in range(num_rows)])
        gradient_2 = 1.0/num_rows * sum([(predicted_value_final(x,i,betas) - y[i])*x[1][i] for i in range(num_rows)]) 
        gradient_3 = 1.0/num_rows * sum([(predicted_value_final(x,i,betas) - y[i])*x[2][i] for i in range(num_rows)])
        gradient_4 = 1.0/num_rows * sum([(predicted_value_final(x,i,betas) - y[i])*x[3][i] for i in range(num_rows)]) 
        
        # Computation of new parameters according to the current gradient.
        temp0 = betas[0] - learning_rate * gradient_0
        temp1 = betas[1] - learning_rate * gradient_1
        temp2 = betas[2] - learning_rate * gradient_2
        temp3 = betas[3] - learning_rate * gradient_3
        temp4 = betas[4] - learning_rate * gradient_4

    
        # Simultaneous Update of Parameters Beta_0 and Beta_1.
        betas[0] = temp0
        betas[1] = temp1
        betas[2] = temp2
        betas[3] = temp3
        betas[4] = temp4


        
        current_error = error(x, y, betas)
        
        if num_iter % 10 == 0:
            print ('Current Value of RSS (Cost Function) based on updated values= ',  current_error)
            
        _error = current_error   # update error 
        num_iter = num_iter + 1  # update iter
    
        if num_iter == maximum_iterations:
            print ("Training Interrupted as Maximum number of iterations were crossed.\n\n")
            converge_status = True
    
    return (betas)

In [180]:
def predict_probability(coef,X):
	beta_0 = coef[0]
	beta_1 = coef[1]
	# Change 11
	beta_2 = coef[2]
	beta_3 = coef[3]
	beta_4 = coef[4]
    
	fy = []
	if len(X) > 1:
		for x in X:
			a = beta_0 + (beta_1 * x[0]) + (beta_2 * x[1]) + (beta_3 * x[2]) + (beta_4 * x[3])
			result = logistic(a)
			print(x[0],x[1],x[2],x[3],a,result)
			fy.append(result)
		return fy

	# Our Regression Model defined using the coefficients from slr function
	x = X[0]
	Y = logistic(beta_0 + (beta_1 * x[0]) + (beta_2 * x[1]) + (beta_3 * x[2]) + (beta_4 * x[3]))

	return Y

In [181]:
def classify(predicted_probability,threshold = 0.5):
    if predicted_probability > threshold:
        return 1
    else:
        return 0

In [182]:
def get_confusion_matrix(test_Y,pred_Y):
    tp = (pred_Y[test_Y == 1])
    tp = tp[tp==1]
    tp = tp.shape[0]
    
    tn = (pred_Y[test_Y == 0])
    tn = tn[tn==0]
    tn = tn.shape[0]
    
    fn = 90 - tn
    fp = 45 - tp
    
    return [tp,fp,tn,fn]


In [183]:
# Based on components of confusion matrix, we compute accuracy, precision and recall evaluation metrics.
def evaluate_performance(test_Y,pred_Y):
    tp,fp,tn,fn = get_confusion_matrix(test_Y,pred_Y)
    
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # Precision of Positive class = (TP / TP + FP) 
    # Similarly precision of negative class = (TN / TN + FN)
    # Precision = Avg (Precision of Positive Class, Precision of Negative Class)
    precision = ((tp / (tp + fp)) + (tn / (tn + fn)))/2
    
    # Recall of Positive class = (TP / TP + FN) 
    # Similarly Recall of Negative class = (TN / TN + FP)
    # Recall = Avg (Recall of Positive Class, Recall of Negative Class)
    recall = ((tp/(tp+fn)) + (tn/(tn+fp)))/2
    
    return [round(accuracy*100,2),round(precision*100,2),round(recall*100,2)]

In [184]:
X,Y,mean,std = get_data("iris.csv")

train_X = np.concatenate((X[:,0:5],X[:,50:55],X[:,100:105]),1)
#print(train_X)
test_X = np.concatenate((X[:,5:50],X[:,55:100],X[:,105:150]),1)
##print(test_X)

train_Y = np.concatenate((Y[0:5],Y[50:55],Y[100:105]),0)
test_Y = np.concatenate((Y[5:50],Y[55:100],Y[105:150]),0)


################################################
## Model Training (or coefficient estimation) ##
################################################
# Using our gradient descent function we estimate coefficients of our regression line. The gradient descent function 
# returns a list of coefficients

coefficients = gradientDescentAlgorithm(train_X,train_Y,0.1)

########################
## Making Predictions ##
########################

# Using our predict function and the coefficients given by our gradient descent function we can now predict the time it will take
# for the next eruption.
print ("Final Values for Beta Parameters are (from beta_0 to beta_4) :",coefficients)



Training Linear Regression Model using Gradient Descent
('Current Value of RSS (Cost Function) based on updated values= ', 11.542839946499335)
('Current Value of RSS (Cost Function) based on updated values= ', 7.7123975312048945)
('Current Value of RSS (Cost Function) based on updated values= ', 5.8736486902746625)
('Current Value of RSS (Cost Function) based on updated values= ', 4.812591291653202)
('Current Value of RSS (Cost Function) based on updated values= ', 4.115335565632171)
('Current Value of RSS (Cost Function) based on updated values= ', 3.6173777989538944)
('Current Value of RSS (Cost Function) based on updated values= ', 3.241175113990322)
('Current Value of RSS (Cost Function) based on updated values= ', 2.9452963658621387)
('Current Value of RSS (Cost Function) based on updated values= ', 2.7054867172557127)
('Current Value of RSS (Cost Function) based on updated values= ', 2.5065370845406134)
Training Interrupted as Maximum number of iterations were crossed.


('Final 

In [185]:
pred_Y = []
for i in range(0,np.transpose(test_X).shape[0]):
    probability = predict_probability(coefficients,[np.transpose(test_X)[i]])
    pred_Y.append(classify(probability))
    
pred_Y = np.array(pred_Y)
    
accuracy,precision,recall = evaluate_performance(test_Y,pred_Y)

print ("Accuracy:",accuracy,"Precision:",precision,"Recall:",recall)


('Accuracy:', 0.0, 'Precision:', 0.0, 'Recall:', 0.0)
